In [1]:
# take too long to load embedding and match it with pid in each author group, 
# preprocess the embedding speed up the process
import os
# load labeled data pid
Dataset = "pubmed"
fileDir = "Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)
pid_need_extract = []

for file in listfiles:
    if not file.startswith('.'):
        with open(fileDir+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                # some record's doi contain \r or \n character in which creating issue, since we do not use those, ignore it
                if(len(read_data)==13 or len(read_data)==12):
                    pid_need_extract.append(int(read_data[0]))
                else:
                    print(len(read_data))
        f.close()
print("Total labeled records:",len(pid_need_extract))

Total labeled records: 140266


In [2]:
# extract 135796 embedding out of all embedding
pid_need_extract = list(sorted(set(pid_need_extract)))
print("Unique pid: ", len(pid_need_extract))

Unique pid:  135796


In [3]:
# extract text info based on paperID and generate new smaller file with id, title, abstract, keyword, and mesh
filePath = "Data/"+Dataset+"/id_text_combined.txt"
all_text = []
num_fail_extract = 0

with open(filePath, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split("\t")
        all_text.append(read_data)
f.close()
print(all_text[0])
print(all_text[0][0])
print("Total records:",len(all_text))

['27406695', 'A comparison of hemagglutination inhibition and neutralization assays for characterizing immunity to seasonal influenza A', 'cross-protection$$hemagglutination inhibition test$$immunity$$influenza$$microneutralization test$$neutralization test', '', "Serum antibody to influenza can be used to identify past exposure and measure current immune status. The two most common methods for measuring this are the hemagglutination inhibition assay (HI) and the viral neutralization assay (NT), which have not been systematically compared for a large number of influenza viruses. A total of 151 study participants from near Guangzhou, China, were enrolled in 2009 and provided serum. HI and NT assays were performed for 12 historic and recently circulating strains of seasonal influenza A. We compared titers using Spearman correlation and fit models to predict NT using HI results. We observed high positive mean correlation between HI and NT assays (Spearman's rank correlation, ?=.86) across

In [4]:
print(len(pid_need_extract))
print(len(all_text))

135796
3151504


In [5]:
all_text = sorted(all_text,key=lambda x: (int(x[0])))
print(len(all_text))
print(all_text[:3])

3151504
[['3', 'Metal substitutions incarbonic anhydrase: a halide ion probe study', '', 'animals$$binding sites$$cadmium$$carbonic anhydrases$$cattle$$humans$$hydrogen-ion concentration$$magnetic resonance spectroscopy$$mercury$$protein binding$$protein conformation$$zinc', '\n'], ['46', 'Purification and properties of Escherichia coli dihydrofolate reductase', '', 'amino acids$$chromatography, affinity$$escherichia coli$$hydrogen-ion concentration$$isoenzymes$$kinetics$$macromolecular substances$$methotrexate$$molecular weight$$osmolar concentration$$potassium chloride$$protein conformation$$sodium chloride$$tetrahydrofolate dehydrogenase', 'Dihydrofolate reductase has been purified 40-fold to apparent homogeneity from a trimethoprim-resistant strain of Escherichia coli (RT 500) using a procedure that includes methotrexate affinity column chromatography. Determinations of the molecular weight of the enzyme based on its amino acid composition, sedimentation velocity, and sodium dodecy

In [6]:
# extract needed embedding
counter = 0
labeled_text_list = []
num_fail_extract = 0
for labeled_text in all_text:
    if(len(pid_need_extract)==0):
        break
    while (pid_need_extract[0]<=int(labeled_text[0])):
        if pid_need_extract[0]==int(labeled_text[0]):
            labeled_text_list.append(labeled_text)
            pid_need_extract.remove(int(labeled_text[0]))
        elif (pid_need_extract[0]<int(labeled_text[0])):
            # remove paper that not in all dataset
            print(pid_need_extract[0], " : ",labeled_text[0])
            pid_need_extract.remove(pid_need_extract[0])
            num_fail_extract+=1
        if len(pid_need_extract)==0:
            break
print("Total paper fail to find: ",num_fail_extract)

Total paper fail to find:  0


In [7]:
print(len(labeled_text_list))
print(labeled_text_list[:3])

135796
[['8077', 'Energetics of primary processes in visula escitation: photocalorimetry of rhodopsin in rod outer segment membranes', '', 'animals$$binding sites$$calorimetry$$cattle$$ferric compounds$$hydrogen-ion concentration$$hydroxylamines$$light$$lysine$$models, biological$$oxalates$$photochemistry$$photoreceptor cells$$radiation effects$$retinal pigments$$retinaldehyde$$rhodopsin$$temperature$$thermodynamics$$vision, ocular', 'A sensitive technique for the direct calorimetric determination of the energetics of photochemical reactions under low levels of illumination, and its application to the study of primary processes in visula excitation, are described. Enthlpies are reported for various steps in the bleaching of rhodopsin in intact rod outer segment membranes, together with the heats of appropriate model reactions. Protonation changes are also determined calorimetrically by use of buffers with differing heats of proton ionization. Bleaching of rhodopsin is accompanied by si

In [10]:
# export needed embedding to txt file
newfileDir = "Data/"+Dataset+"/"
if not os.path.exists(newfileDir):
    os.makedirs(newfileDir)
    
with open(newfileDir+"id_text_combined_labeled.txt", "w",encoding='utf8') as f:
    for feature in labeled_text_list:
        f.write(("\t".join(feature)).strip('\n')+"\n")
f.close()